##### Copyright 2020 The TensorFlow Authors.

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# 희소 텐서 작업

<table class="tfo-notebook-buttons" align="left">
  <td><a target="_blank" href="https://www.tensorflow.org/guide/sparse_tensor"><img src="https://www.tensorflow.org/images/tf_logo_32px.png">TensorFlow.org에서 보기</a></td>
  <td><a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs/blob/master/site/en/guide/sparse_tensor.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png">Google Colab에서 실행</a></td>
  <td><a target="_blank" href="https://github.com/tensorflow/docs/blob/master/site/en/guide/sparse_tensor.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png">GitHub에서 보기</a></td>
  <td><a href="https://storage.googleapis.com/tensorflow_docs/docs/site/en/guide/sparse_tensor.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png">노트북 다운로드</a></td>
</table>

0 값이 많이 포함된 텐서로 작업할 때 공간 및 시간 효율적인 방식으로 저장하는 것이 중요합니다. 희소 텐서는 0 값이 많이 포함된 텐서를 효율적으로 저장하고 처리할 수 있도록 합니다. [스파스 텐서는 NLP 애플리케이션에서 데이터 사전 처리의 일부로 TF-IDF](https://en.wikipedia.org/wiki/Tf%E2%80%93idf) 와 같은 인코딩 방식과 컴퓨터 비전 애플리케이션에서 어두운 픽셀이 많은 이미지를 사전 처리하는 데 광범위하게 사용됩니다.

## TensorFlow의 희소 텐서

`tf.SparseTensor` 객체를 통해 희소 텐서를 나타냅니다. 현재 TensorFlow의 희소 텐서는 좌표 목록(COO) 형식을 사용하여 인코딩됩니다. 이 인코딩 형식은 임베딩과 같은 초희소 행렬에 최적화되어 있습니다.

희소 텐서에 대한 COO 인코딩은 다음으로 구성됩니다.

- `values` : 0이 아닌 모든 값을 포함하는 `[N]` 모양의 1D 텐서.
- `indices` : 0이 아닌 값의 인덱스를 포함하는 `[N, rank]` 모양의 2D 텐서.
- `dense_shape` : 텐서의 모양을 지정하는 `[rank]` 모양의 1D 텐서.

`tf.SparseTensor` 컨텍스트에서 ***0이 아닌*** 값은 명시적으로 인코딩되지 않은 값입니다. 이는 명시 적으로 제로 값을 포함 할 수있다 `values` COO 희소 행렬의, 그러나 성긴 텐서에 제로 값을 참조 할 때 이러한 "명시 제로"는 일반적으로 포함되지 않는다.

참고: `tf.SparseTensor` 는 인덱스/값의 특정 순서를 요구하지 않지만 여러 작업에서는 행 우선 순위로 가정합니다. `tf.sparse.reorder` 를 사용하여 표준 행 우선 순서로 정렬된 희소 텐서의 복사본을 만듭니다. 

## `tf.SparseTensor` 만들기

`values` , `indices` 및 `dense_shape` 를 직접 지정하여 희소 텐서를 구성합니다.

In [ ]:
import tensorflow as tf

In [ ]:
st1 = tf.SparseTensor(indices=[[0, 3], [2, 4]],
                      values=[10, 20],
                      dense_shape=[3, 10])

<img src="images/sparse_tensor.png">

희소 텐서를 인쇄하기 위해 `print()` 함수를 사용하면 세 가지 구성 요소 텐서의 내용이 표시됩니다.

In [ ]:
print(st1)

`values` `indices` 와 정렬되어 있으면 희소 텐서의 내용을 이해하는 것이 더 쉽습니다. 0이 아닌 각 값이 자체 줄에 표시되도록 희소 텐서를 예쁘게 인쇄하는 도우미 함수를 정의합니다.

In [ ]:
def pprint_sparse_tensor(st):
  s = "<SparseTensor shape=%s \n values={" % (st.dense_shape.numpy().tolist(),)
  for (index, value) in zip(st.indices, st.values):
    s += f"\n  %s: %s" % (index.numpy().tolist(), value.numpy().tolist())
  return s + "}>"

In [ ]:
print(pprint_sparse_tensor(st1))

또한 사용하여 밀도 텐서에서 스파 스 텐서를 구성 할 수 `tf.sparse.from_dense` 하고 사용하여 조밀 한 텐서로 다시 변환 `tf.sparse.to_dense` .

In [ ]:
st2 = tf.sparse.from_dense([[1, 0, 0, 8], [0, 0, 0, 0], [0, 0, 3, 0]])
print(pprint_sparse_tensor(st2))

In [ ]:
st3 = tf.sparse.to_dense(st2)
print(st3)

## 희소 텐서 조작

`tf.sparse` 패키지의 유틸리티를 사용하여 희소 텐서를 조작하십시오. 조밀한 텐서의 산술 조작에 사용할 수 있는 `tf.math.add` 와 같은 연산은 희소 텐서에서는 작동하지 않습니다.

`tf.sparse.add` 를 사용하여 동일한 모양의 희소 텐서를 추가합니다. 

In [ ]:
st_a = tf.SparseTensor(indices=[[0, 2], [3, 4]],
                       values=[31, 2], 
                       dense_shape=[4, 10])

st_b = tf.SparseTensor(indices=[[0, 2], [7, 0]],
                       values=[56, 38],
                       dense_shape=[4, 10])

st_sum = tf.sparse.add(st_a, st_b)

print(pprint_sparse_tensor(st_sum))

`tf.sparse.sparse_dense_matmul` 을 사용하여 희소 텐서를 고밀도 행렬과 곱합니다.

In [ ]:
st_c = tf.SparseTensor(indices=([0, 1], [1, 0], [1, 1]),
                       values=[13, 15, 17],
                       dense_shape=(2,2))

mb = tf.constant([[4], [6]])
product = tf.sparse.sparse_dense_matmul(st_c, mb)

print(product)

tf.sparse.concat 을 사용하여 희소 텐서를 함께 `tf.sparse.concat` 를 사용하여 `tf.sparse.slice` 합니다.


In [ ]:
sparse_pattern_A = tf.SparseTensor(indices = [[2,4], [3,3], [3,4], [4,3], [4,4], [5,4]],
                         values = [1,1,1,1,1,1],
                         dense_shape = [8,5])
sparse_pattern_B = tf.SparseTensor(indices = [[0,2], [1,1], [1,3], [2,0], [2,4], [2,5], [3,5], 
                                              [4,5], [5,0], [5,4], [5,5], [6,1], [6,3], [7,2]],
                         values = [1,1,1,1,1,1,1,1,1,1,1,1,1,1],
                         dense_shape = [8,6])
sparse_pattern_C = tf.SparseTensor(indices = [[3,0], [4,0]],
                         values = [1,1],
                         dense_shape = [8,6])

sparse_patterns_list = [sparse_pattern_A, sparse_pattern_B, sparse_pattern_C]
sparse_pattern = tf.sparse.concat(axis=1, sp_inputs=sparse_patterns_list)
print(tf.sparse.to_dense(sparse_pattern))

In [ ]:
sparse_slice_A = tf.sparse.slice(sparse_pattern_A, start = [0,0], size = [8,5])
sparse_slice_B = tf.sparse.slice(sparse_pattern_B, start = [0,5], size = [8,6])
sparse_slice_C = tf.sparse.slice(sparse_pattern_C, start = [0,10], size = [8,6])
print(tf.sparse.to_dense(sparse_slice_A))
print(tf.sparse.to_dense(sparse_slice_B))
print(tf.sparse.to_dense(sparse_slice_C))

TensorFlow 2.4 이상을 사용하는 경우 희소 텐서의 0이 아닌 값에 대한 요소별 작업에 `tf.sparse.map_values`

In [ ]:
st2_plus_5 = tf.sparse.map_values(tf.add, st2, 5)
print(tf.sparse.to_dense(st2_plus_5))

0이 아닌 값만 수정되었다는 점에 유의하십시오. 0 값은 0으로 유지됩니다.

마찬가지로 이전 버전의 TensorFlow에 대해 아래 디자인 패턴을 따를 수 있습니다.

In [ ]:
st2_plus_5 = tf.SparseTensor(
    st2.indices,
    st2.values + 5,
    st2.dense_shape)
print(tf.sparse.to_dense(st2_plus_5))

## 다른 TensorFlow API와 함께 `tf.SparseTensor` 사용

희소 텐서는 다음 TensorFlow API와 투명하게 작동합니다.

- `tf.keras`
- `tf.data`
- `tf.Train.Example` protobuf
- `tf.function`
- `tf.while_loop`
- `tf.cond`
- `tf.identity`
- `tf.cast`
- `tf.print`
- `tf.saved_model`
- `tf.io.serialize_sparse`
- `tf.io.serialize_many_sparse`
- `tf.io.deserialize_many_sparse`
- `tf.math.abs`
- `tf.math.negative`
- `tf.math.sign`
- `tf.math.square`
- `tf.math.sqrt`
- `tf.math.erf`
- `tf.math.tanh`
- `tf.math.bessel_i0e`
- `tf.math.bessel_i1e`

위의 API 중 일부에 대한 예가 아래에 나와 있습니다.

### `tf.keras`

`tf.keras` API의 하위 집합은 값비싼 캐스팅 또는 변환 작업 없이 희소 텐서를 지원합니다. Keras API를 사용하면 희소 텐서를 Keras 모델에 대한 입력으로 전달할 수 있습니다. `tf.keras.Input` 또는 `tf.keras.layers.InputLayer` 호출할 때 `sparse=True` 설정합니다. Keras 계층 간에 희소 텐서를 전달할 수 있으며 Keras 모델이 이를 출력으로 반환하도록 할 수도 있습니다. `tf.keras.layers.Dense` 레이어에서 희소 텐서를 사용하면 밀도가 높은 텐서를 출력합니다.

아래 예는 희소 입력을 지원하는 계층만 사용하는 경우 희소 텐서를 Keras 모델에 대한 입력으로 전달하는 방법을 보여줍니다.

In [ ]:
x = tf.keras.Input(shape=(4,), sparse=True)
y = tf.keras.layers.Dense(4)(x)
model = tf.keras.Model(x, y)

sparse_data = tf.SparseTensor(
    indices = [(0,0),(0,1),(0,2),
               (4,3),(5,0),(5,1)],
    values = [1,1,1,1,1,1],
    dense_shape = (6,4)
)

model(sparse_data)

model.predict(sparse_data)

### `tf.data`

`tf.data` API를 사용하면 간단하고 재사용 가능한 부분에서 복잡한 입력 파이프라인을 구축할 수 있습니다. 핵심 데이터 구조는 각 요소가 하나 이상의 구성 요소로 구성된 요소 시퀀스를 나타내는 `tf.data.Dataset`

#### 희소 텐서를 사용하여 데이터세트 빌드

`tf.Tensor` `tf.data.Dataset.from_tensor_slices` 또는 NumPy 배열에서 구축하는 데 사용되는 것과 동일한 방법을 사용하여 희소 텐서에서 데이터세트를 구축합니다. 이 연산은 데이터의 희소성(또는 희소성)을 유지합니다.

In [ ]:
dataset = tf.data.Dataset.from_tensor_slices(sparse_data)
for element in dataset: 
  print(pprint_sparse_tensor(element))

#### 희소 텐서를 사용한 데이터 세트 일괄 처리 및 일괄 해제

`Dataset.batch` 및 `Dataset.unbatch` 메서드를 각각 사용하여 희소 텐서를 사용하여 데이터 세트를 일괄 처리(연속 요소를 단일 요소로 결합)하고 일괄 처리 해제할 수 있습니다.

In [ ]:
batched_dataset = dataset.batch(2)
for element in batched_dataset:
  print (pprint_sparse_tensor(element))

In [ ]:
unbatched_dataset = batched_dataset.unbatch()
for element in unbatched_dataset:
  print (pprint_sparse_tensor(element))

`tf.data.experimental.dense_to_sparse_batch` 를 사용하여 다양한 모양의 데이터 세트 요소를 희소 텐서로 일괄 처리할 수도 있습니다. 

#### 희소 텐서를 사용하여 데이터 세트 변환

`Dataset.map` 사용하여 Datasets에서 희소 텐서를 변환하고 생성합니다.

In [ ]:
transform_dataset = dataset.map(lambda x: x*2)
for i in transform_dataset:
  print(pprint_sparse_tensor(i))

### tf.train.Example

`tf.train.Example` 은 TensorFlow 데이터에 대한 표준 protobuf 인코딩입니다. `tf.train.Example` 과 함께 희소 텐서를 사용할 때 다음을 수행할 수 있습니다.

- `tf.SparseTensor` 사용하여 가변 길이 데이터를 `tf.io.VarLenFeature` . `tf.io.RaggedFeature` 사용을 고려해야 합니다.

- (A) 내로 임의 희소 데이터 읽기 `tf.SparseTensor` 사용 `tf.io.SparseFeature` 저장소에 세 개의 기능 키를 사용하여, `indices` , `values` 및 `dense_shape` .

### `tf.function`

`tf.function` 데코레이터는 Python 함수에 대한 TensorFlow 그래프를 미리 계산하여 TensorFlow 코드의 성능을 크게 향상시킬 수 있습니다. `tf.function` 및 [구체 함수](https://www.tensorflow.org/guide/function#obtaining_concrete_functions) 모두에서 투명하게 작동합니다.

In [ ]:
@tf.function
def f(x,y):
  return tf.sparse.sparse_dense_matmul(x,y)

a = tf.SparseTensor(indices=[[0, 3], [2, 4]],
                    values=[15, 25],
                    dense_shape=[3, 10])

b = tf.sparse.to_dense(tf.sparse.transpose(a))

c = f(a,b)

print(c)

## 0 값과 결측값 구별하기

tf.SparseTensor 에 대한 대부분의 연산은 `tf.SparseTensor` 과 명시적 0 값을 동일하게 취급합니다. 이것은 의도적으로 `tf.SparseTensor` 입니다. tf.SparseTensor는 조밀한 텐서처럼 작동해야 합니다.

그러나 0 값과 결측값을 구별하는 것이 유용할 수 있는 몇 가지 경우가 있습니다. 특히, 이것은 훈련 데이터에서 누락/알 수 없는 데이터를 인코딩하는 한 가지 방법을 허용합니다. 예를 들어, 일부 누락된 점수와 함께 점수의 텐서(-Inf에서 +Inf까지의 모든 부동 소수점 값을 가질 수 있음)가 있는 사용 사례를 고려하십시오. 명시적 0은 알려진 0 점수이지만 암시적 0 값은 실제로 0이 아닌 누락된 데이터를 나타내는 희소 텐서를 사용하여 이 텐서를 인코딩할 수 있습니다.

참고: 이것은 일반적으로 `tf.SparseTensor` 의 의도된 용도가 아닙니다. 예를 들어 알려진/알 수 없는 값의 위치를 식별하는 별도의 마스크 텐서를 사용하는 것과 같이 이를 인코딩하기 위한 다른 기술도 고려하고 싶을 수 있습니다. 그러나 대부분의 희소 연산은 명시적 및 암시적 0 값을 동일하게 취급하므로 이 접근 방식을 사용하는 동안 주의하십시오.

`tf.sparse.reduce_max` 와 같은 일부 연산은 누락된 값을 0인 것처럼 취급하지 않습니다. 예를 들어 아래 코드 블록을 실행할 때 예상되는 출력은 `0` 입니다. 그러나 이 예외로 인해 출력은 `-3` 입니다.

In [ ]:
print(tf.sparse.reduce_max(tf.sparse.from_dense([-5, 0, -3])))

대조적으로 `tf.math.reduce_max` 를 조밀한 텐서에 적용하면 예상대로 출력이 0이 됩니다.

In [ ]:
print(tf.math.reduce_max([-5, 0, -3]))

## 추가 읽을거리 및 리소스

- 텐서에 대해 알아보려면 [텐서 가이드](https://www.tensorflow.org/guide/tensor) 를 참조하세요.
- 비정형 텐서 가이드를 읽고 비균일 데이터로 작업할 수 있는 텐서 유형인 [비정형 텐서를 사용하는 방법을 알아보세요.](https://www.tensorflow.org/guide/ragged_tensor)
- [`tf.Example` 데이터 디코더](https://github.com/tensorflow/models/blob/9139a7b90112562aec1d7e328593681bd410e1e7/research/object_detection/data_decoders/tf_example_decoder.py) 에서 희소 텐서를 사용 [하는 TensorFlow Model Garden](https://github.com/tensorflow/models) 에서 이 객체 감지 모델을 확인하십시오.
